Sparse Categorical CrossEntropy

In [ ]:
!pip install transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

Training Data:

In [ ]:
#from sklearn.datasets import fetch_20newsgroups
#categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
#newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
#newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
newsgroups_train = fetch_20newsgroups(data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True, return_X_y=False)
newsgroups_test = fetch_20newsgroups(data_home=None, subset='test', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True, return_X_y=False)
data = newsgroups_train.data
targets = newsgroups_train.target
categories = newsgroups_train.target_names
print(set(targets))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


In [ ]:
# Training Data
# about 2 min
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

dict_data = tokenizer(data, return_tensors="tf", truncation=True, padding=True)
dict_data["labels"] = tf.one_hot(targets, 20)


In [ ]:
# Instantiate from HuggingFace

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=20)

model.summary()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  15380     
Total params: 109,497,620
Trainable params: 109,497,620
Non-trainable params: 0
_________________________________________________________________


In [ ]:

optimizer = tf.keras.optimizers.Adam()
#optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
model_history = model.fit( [dict_data["input_ids"], dict_data["attention_mask"], dict_data["token_type_ids"]], [dict_data["labels"]], verbose=1, batch_size=4, epochs=3)

Epoch 1/3
2829/2829 [==============================] - 1531s 541ms/step - loss: 0.8423 - accuracy: 0.7550
Epoch 2/3
2829/2829 [==============================] - 1533s 542ms/step - loss: 0.2885 - accuracy: 0.9187
Epoch 3/3
2829/2829 [==============================] - 1536s 543ms/step - loss: 0.1609 - accuracy: 0.9586
